# 2x2 Game Equilibria Algorithms

In [1]:
import numpy as np

In [165]:
#Stag hunt - p.49
a1 = 5; b1 = 0; c1 = 4; d1 = 2
a2 = 5; b2 = 4; c2 = 0; d2 = 2

P1 = np.array([[a1, b1],
              [c1, d1]])

P2 = np.array([[a2, b2],
               [c2, d2]])

## Pure strategy Algorithm

In `pure` we tally up how many of the players have a certain square as their "preferred outcome".

In [166]:
pure = np.array([[0, 0],
                 [0, 0]])

First, we check which row P1 prefers for each coice of column P2 makes.

In [167]:
for i in range(2):
    col = P1[:,i]
    if col[0] > col[1]:
        pure[0,i] += 1
    elif col[1] > col[0]:
        pure[1,i] += 1
    else:
        pure[:,i] += 1

Then, we check which column P2 prefers for each coice of row P2 makes.

In [168]:
for i in range(2):
    row = P2[i,:]
    if row[0] > row[1]:
        pure[i,0] += 1
    elif row[1] > row[0]:
        pure[i,1] += 1
    else:
        pure[i,:] += 1

Wherever we have the number 2 in `pure` we have a nash equilibrium.

In [169]:
print(pure)

[[2 0]
 [0 2]]


We can put the entire algorithm into a function.

In [170]:
def pureAlgo(P1, P2):
    pure = np.array([[0, 0], [0, 0]])
    for i in range(2):
        col = P1[:,i]
        if col[0] > col[1]:
            pure[0,i] += 1
        elif col[1] > col[0]:
            pure[1,i] += 1
        else:
            pure[:,i] += 1
    
    for i in range(2):
        row = P2[i,:]
        if row[0] > row[1]:
            pure[i,0] += 1
        elif row[1] > row[0]:
            pure[i,1] += 1
        else:
            pure[i,:] += 1
    
    return(pure)

## Mixed stragety algorithm

Finding a mixed strategy in a 2x2 game is more algebra than following a proper algorithm. The idea is to find what propability P1 must play row 1 and row 2 to make P2 indifferend between column 1 and column 2. And vice versa.

We have a game matrix

|       | Left | Right |
| ----- | -------- |--------- |
| Up | a1 / a2  |b1 / b2   |
| Down | c1 / c2  |d1 / d2   |

We want the expected utility of left $EU_L$ to be the same as the expected utility of right $EU_R$, so that P2 is indifferent between the two. The probability of P1 playing up $P_U$ that satisfies this constraint can be found by these three equations:

$$
\begin{aligned}
EU_L &= EU_R \\
EU_L &= P_U a_2 + (1 - P_U) c_2 \\
EU_R &= P_U b_2 + (1 - P_U) d_2
\end{aligned}
$$

Make sure you see how we found the expected utilities. We now get:

This might look complicated, but you should be able to follow the steps if you remember the rules for solving simple equations like 6x + 6 = 20 - x

$$
\begin{aligned}
EU_L &= EU_R \\
P_U  a_2 + (1 - P_U) c_2 &= P_U b_2 + (1 - P_U) d_2 \\
P_U a_2 + c2 - P_U c2 &= P_U b_2 + d2 - P_U d_2 \\
P_U a_2 - P_U c_2 - P_U b_2 + P_U d_2 &= -c2 + d_2 \\
P_U (a_2 - b_2 - c_2 + d_2) &= -c_2 + d_2 \\
P_U &= \frac{-c_2 + d_2}{a_2 - b_2 - c_2 + d_2}
\end{aligned}
$$

We now have an expression that gives us the probability of playing up in the mixed strategy that makes P2 indifferent between playing left and right. The probability of playing down is of course $1 - P_U$. The probability of playing left and right that makes P1 indifferent between playing up and down can be found with the same method. You should try it to make sure you understood the steps.

When P1 and P2 both play these strategies they are both indifferent between their choices, meaning they have no incentive to change their strategy, and they are playing a nash equilibrium!

We can put the expressions for $P_U$ and $P_L$ into a function for finding the mixed stragety.

In [171]:
def mixedAlgo(P1, P2):
    #Unpacking the matrixes
    a1 = P1[0,0]; b1 = P1[0,1]; c1 = P1[1,0]; d1 = P1[1,1]
    a2 = P2[0,0]; b2 = P2[0,1]; c2 = P2[1,0]; d2 = P2[1,1]
    
    if (a2 - b2 - c2 + d2) == 0 or (a1 - b1 - c1 + d1) == 0: #Zero division check
        return
    
    row1 = (-c2 + d2) / (a2 - b2 - c2 + d2)
    col1 = (-b1 + d1) / (a1 - b1 - c1 + d1)
    return [row1, 1 - row1] , [col1, 1 - col1]

We can test it on the stag hunt from before:

In [172]:
print(mixedAlgo(P1, P2))

([0.6666666666666666, 0.33333333333333337], [0.6666666666666666, 0.33333333333333337])


The mixed strategy equilibrium for the stag hunt is for P1 to play row 1 2/3 of the time, and for P2 to play column 1 2/3 of the time.

### Limitations of the mixed strategy algorithm

If the mixed strategy contains probabilities above 1 or below 0 you must discard it. You must also make sure you do not divide by zero inside the algorithm, as that will produce an error. If you would divide by zero, there is no mixed equilibria. This algorithm does not find eqilibria where one player plays a pure strategy and another plays a mixed strategy. If two cells in the same row or the same column are pure strategy nash equilibria, they also form an infinite number of mixed strategy nash equilibra, since one player can play only that row(or column) while the other player mixes between the two pure strategies that are parth of the equilibria.